# Investigation of performance of net mechanism on a two-dimensional toy data set

In [1]:
import second_moment_experiments_main as experiment
import pickle
import matplotlib
import matplotlib.pyplot as plt
plt.rc('text', usetex = True)
plt.rc('font', family = 'serif')
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
from exputils import extract_data, initialise_netmech_containers, initialise_adassp_reg_containers, \
initialise_netmech_reg_containers, get_expected_statistics, get_optimal_utilities_statistics
from baselines import Regression, DPRegression
import numpy as np
# %matplotlib tk

In [ ]:
# Load data 
exp_name = 'exp_vary_private_data_extended'
with open ('D:/Thesis/Experiments/exp_vary_private_data_extended/' + exp_name, "rb") as container:
    results = pickle.load(container)

## Assess the performance of the netmechanism for sparse lattices for a wide range of datasets

$\textbf{Setup}$: Draw $25$ samples with the net mechanism set up with $k_f = k_t \in \{3:20\}$ for $100$ different data sets.  Compare the expected average RMSE across the $100$ datasets with the results obtained originally for the seed = 23 case. Then compute the expected average RMSE across the $100$ datasets for the ADASSP algorithm and compare the two.

### Parameter setup

In [2]:
# Default parameters list
dimensionality = 2
num_records = 300
test_frac = 0.8
batch_size = 1000
directory = 'D:/Thesis/Experiments/exp_vary_private_data_extended/'
parallel = True
save_data = False
partition_method = 'fast_2'
workers = 5
num_samples = 25
sample_parallel = False 
load_data = False

In [3]:
# Experiment specific
num_points_min = 3
min_seed = 8
num_points_max = 20
num_points_features_vec = range(num_points_min, num_points_max + 1)
num_points_targets_vec = range(num_points_min, num_points_max + 1)
num_datasets = 100
epsilon_vec = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

### Run the experiments

In [4]:
# Initialise results data containers
results = {key_outer: {key_inner : [] for key_inner in epsilon_vec}  for key_outer in range(num_datasets)}

In [ ]:
# Collect results
for dataset in range(min_seed, num_datasets):
    print ("Dataset number", dataset)
    for epsilon in epsilon_vec:
        for num_points_features, num_points_targets in zip(num_points_features_vec, num_points_targets_vec):
            results[dataset][epsilon].append(experiment.second_order_moment_experiment(dimensionality = dimensionality, num_records = num_records, \
                                                                                       test_frac = test_frac, batch_size = batch_size,directory = directory, \
                                                                                       parallel = parallel, save_data = save_data,\
                                                                                       partition_method = partition_method, workers = workers, \
                                                                                       num_samples = num_samples, sample_parallel = sample_parallel,\
                                                                                       load_data = load_data, num_points_targets = num_points_targets,\
                                                                                       num_points_features = num_points_features, epsilon = epsilon, \
                                                                                       seed = dataset))

Dataset number 8
Initialising synthetic feature space lattice
Synthetic feature space initialised
Initialising synthethic target space lattice
Synthethic target space initialised
Number of batches is 1
Number of batches is  1
Starting parallel pool
Partition function is 26.01312045792935
Max scaled utility is -0.6151056537961627
Generation time 2.0469624996185303
Sampling time 0.0
Overall max utility -0.41007043586410846
Elapsed time with 5 workers is 2.050963878631592
Initialising synthetic feature space lattice
Synthetic feature space initialised
Initialising synthethic target space lattice
Synthethic target space initialised
Number of batches is 1
Number of batches is  1
Starting parallel pool
Partition function is 50.928407966088
Max scaled utility is -0.11826015132702852
Generation time 2.1458499431610107
Sampling time 0.0
Overall max utility -0.07884010088468568
Elapsed time with 5 workers is 2.1458499431610107
Initialising synthetic feature space lattice
Synthetic feature space 

Partition function is 8981418.96487145
Max scaled utility is -0.018689008693295418
Generation time 5.936959266662598
Sampling time 2.7074992656707764
Overall max utility -0.012459339128863612
Elapsed time with 5 workers is 8.644458532333374
Initialising synthetic feature space lattice
Synthetic feature space initialised
Initialising synthethic target space lattice
Synthethic target space initialised
Number of batches is 1
Number of batches is  1
Starting parallel pool
Partition function is 11.990707645822928
Max scaled utility is -1.2302113075923253
Generation time 2.095672130584717
Sampling time 0.0
Overall max utility -0.41007043586410846
Elapsed time with 5 workers is 2.0996809005737305
Initialising synthetic feature space lattice
Synthetic feature space initialised
Initialising synthethic target space lattice
Synthethic target space initialised
Number of batches is 1
Number of batches is  1
Starting parallel pool
Partition function is 36.69941542912441
Max scaled utility is -0.2365

In [ ]:
# Save the results for latter access
exp_name = 'exp_vary_private_data_extended'
with open (directory + exp_name, "wb") as container:
    pickle.dump(results, container)

## Analyse the data

In [ ]:
# Experimental data containers
max_utilities, avg_samples_utility, synthetic_datasets_vec, test_set, private_data = \
    initialise_netmech_containers(epsilon_vec, multiple_datasets = True, max_dataset = num_datasets)

In [ ]:
# Extract data from the results data structure 
for dataset_number in results.keys():
    # Quantities that are epsilon independent 
    fixed_eps = 0.1
    max_utilities[dataset_number], test_set[dataset_number],private_data[dataset_number] = extract_data(results[dataset_number][fixed_eps],\
                                                                                                    multiple_datasets = True, \
                                                                                                    max_dataset = num_datasets, \
                                                                                                    eps_dependent = False)
    # Quantities that are epsilon independent
    for epsilon in results[dataset_number].keys():
        avg_samples_utility[dataset_number][epsilon], synthetic_datasets_vec[dataset_number][epsilon] = \
           extract_data(results[dataset_number][epsilon], multiple_datasets = True, max_dataset = num_datasets, \
                          eps_dependent = True) 

In [ ]:
# Initialise containers for the regression on the synthetic data sets released with netmechanism
net_mech_reg_coefs, predictive_errs_netmech, min_predictive_errs_netmech, mean_predictive_errs_netmech,\
double_std_predictive_errs_netmech = initialise_netmech_reg_containers(epsilon_vec, multiple_datasets = True,\
                                                                        max_dataset = num_datasets)

# Initialise containers for the regression on the synthetic data sets with parameters released by ADASSP
adassp_reg_coef, predictive_err_adassp, min_predictive_err_adassp, mean_predictive_err_adassp, double_std_predictive_err_adassp\
                                   = initialise_adassp_reg_containers(epsilon_vec, multiple_datasets = True, max_dataset = num_datasets)

In [ ]:
# Fit ADASSP to the private dataset and calculate the predictive error
for dataset_number in results.keys():
    for epsilon in results[dataset_number].keys():
        adassp_regressor = DPRegression()
        adassp_reg_coef[dataset_number][epsilon] = adassp_regressor.get_parameters(private_data[dataset_number].features, \
                                                                                   private_data[dataset_number].targets,\
                                                                                   num_samples, epsilon, seed = dataset_number)
        predictive_err_adassp[dataset_number][epsilon] = Regression().calculate_predictive_error(private_data[dataset_number].test_data, \
                                                                                                 adassp_reg_coef[dataset_number][epsilon])
        min_predictive_err_adassp[dataset_number][epsilon] = np.min(predictive_err_adassp[dataset_number][epsilon])
        mean_predictive_err_adassp[dataset_number][epsilon] = np.mean(predictive_err_adassp[dataset_number][epsilon])
        double_std_predictive_err_adassp[dataset_number][epsilon] = 2*np.std(predictive_err_adassp[dataset_number][epsilon])

In [ ]:
# Perform regression on the datasets released with the net mechanism and calculate predictive error statistics
for dataset_number in results.keys():
    for epsilon in results[dataset_number].keys():
        for synthetic_datasets in synthetic_datasets_vec[dataset_number][epsilon]:
            netmech_regressor = Regression()
            net_mech_reg_coef = netmech_regressor.fit_data(synthetic_datasets)
            net_mech_reg_coefs[dataset_number][epsilon].append(net_mech_reg_coef)
            predictive_err_netmech = netmech_regressor.calculate_predictive_error(private_data[dataset_number].test_data, \
                                                                                  net_mech_reg_coef)
            predictive_errs_netmech[dataset_number][epsilon].append(predictive_err_netmech)
            min_predictive_errs_netmech[dataset_number][epsilon].append(np.min(predictive_err_netmech))
            mean_predictive_errs_netmech[dataset_number][epsilon].append(np.mean(predictive_err_netmech))
            double_std_predictive_errs_netmech[dataset_number][epsilon].append(2*np.std(predictive_err_netmech))

In [ ]:
# Calculate the desired satistics           
expected_avg_utility, min_avg_utility, max_avg_utility, expected_mean_predictive_errs_netmech, \
expected_double_std_predictive_errs_netmech, expected_mean_predictive_err_adassp, \
expected_double_std_predictive_err_adassp, expected_min_predictive_errs_netmech, \
min_min_predictive_errs_netmech, max_min_predictive_errs_netmech, expected_min_predictive_err_adassp, \
min_min_predictive_err_adassp, max_min_predictive_err_adassp = get_expected_statistics(avg_samples_utility,\
                                                                    mean_predictive_errs_netmech,\
                                                                    double_std_predictive_errs_netmech,\
                                                                    mean_predictive_err_adassp,\
                                                                    double_std_predictive_err_adassp, \
                                                                    min_predictive_errs_netmech, \
                                                                    min_predictive_err_adassp, \
                                                                    epsilon_vec)
expected_optimal_utilities, min_optimal_utilities, max_optimal_utilities = get_optimal_utilities_statistics(max_utilities)

## Plot the results

In [ ]:
xcoordinate = num_points_features_vec
# Define the range of epsilon for which various quntities are plotted
epsilon_range = [0.1, 0.6, 5.0]

#### Expected average samples utility

In [ ]:
def plot_expected_avg_utility(xcoordinate, expected_avg_samples_utility, expected_optimal_utilities, epsilon):
    plt.plot(xcoordinate, expected_avg_samples_utility, 'g--^', label = r'$E[\mu_u]$' )
    plt.plot(xcoordinate, expected_optimal_utilities, 'r*', label = r'$E[opt(u)]$')
    plt.xticks(xcoordinate)
    plt.xlabel('Lattice denisity (points)', fontsize = 20)
    plt.ylabel(r'$\mu_u$ ($\varepsilon =' + str(epsilon) + '$)', fontsize = 20)
    plt.legend(fontsize = 15)
    plt.grid(True)

for epsilon in epsilon_range:
    plot_expected_avg_utility(xcoordinate, expected_avg_utility[epsilon],\
                            expected_optimal_utilities, epsilon)
    plt.plot(xcoordinate, avg_samples_utility[23][epsilon], 'yH', label = r'$\mu_u^{23}$')
    plt.plot(xcoordinate, max_utilities[23], 'cD', label = r'$opt(u)^{23}$')
    plt.legend(fontsize = 15)
    plt.show()
    

#### Variation of average RMSE with $\varepsilon$ for various lattice densities

In [ ]:
# print(xcoordinate)
# print(expected_mean_predictive_errs_netmech)
print(expected_mean_predictive_err_adassp)
print(expect)

In [ ]:
# Calculate variation of RMSE as a function of epsilon
expected_rmse_eps_avg_netmech = {key: [] for key in xcoordinate}
expected_rmse_eps_std_netmech = {key: [] for key in xcoordinate}
expected_rmse_eps_avg_adassp = []
expected_prmse_eps_std_adassp = []

# Extract data
for epsilon in epsilon_vec[0: len(epsilon_vec)]:
    # print ("DEBUG:", mean_predictive_errs_netmech[epsilon])
    # print ("DEBUG:", len(mean_predictive_errs_netmech[epsilon]))
    for index in xcoordinate:
        expected_rmse_eps_avg_netmech[index].append(expected_mean_predictive_errs_netmech[epsilon][index - xcoordinate[0]])
        expected_rmse_eps_std_netmech[index].append(expected_double_std_predictive_errs_netmech[epsilon][index - xcoordinate[0]])

expected_rmse_eps_avg_adassp = [expected_mean_predictive_err_adassp[epsilon] for epsilon in epsilon_vec[0: len(epsilon_vec)]] 
expected_rmse_eps_std_adassp = [expected_double_std_predictive_err_adassp[epsilon] for epsilon in epsilon_vec[0: len(epsilon_vec)]]

def plot_expected_rmse_avg_epsilon(epsilon_vec, expected_rmse_eps_avg_netmech, expected_rmse_eps_avg_adassp, lattice_density,\
                                   adassp = True):
    plt.semilogx(epsilon_vec, expected_rmse_eps_avg_netmech, '--*', label = 'Net mechansim')
    if adassp:
        plt.plot(epsilon_vec, expected_rmse_eps_avg_adassp, '--*', label = 'ADASSP')
        # plt.ylim([0, 1.5]) 
    plt.xlabel(r'$\varepsilon$', fontsize = 18)
    # plt.yticks(np.arange(0,1.5, step = 0.1))
   #  plt.xticks(epsilon_vec)
    plt.ylabel('$E[\mu_{RMSE}]$ (density = ' + str(lattice_density) +  ' points)', fontsize = 18)
    plt.legend(fontsize = 14)
    plt.grid(True)
    plt.show()
        
for lattice_density in xcoordinate:
    # print ("DEBUG, rmse_eps_avg_netmech", rmse_eps_avg_netmech[lattice_density])
    plot_expected_rmse_avg_epsilon(epsilon_vec[0: len(epsilon_vec)], expected_rmse_eps_avg_netmech[lattice_density],\
                                   expected_rmse_eps_avg_adassp, lattice_density, adassp = True)
    
# Plot average RMSE for ADASSP for selected values of the lattice density on the same graph
plt.figure(1)
xcoordinate_range = [3, 4, 5]
#xcoordinate_range = xcoordinate
for lattice_density in xcoordinate_range:
    plt.semilogx(epsilon_vec[0: len(epsilon_vec)], expected_rmse_eps_avg_netmech[lattice_density], '--*', \
                 label = 'density = ' + str(lattice_density))
plt.plot(epsilon_vec[0: len(epsilon_vec)], expected_rmse_eps_avg_adassp,'-*', label = 'ADASSP')
plt.xlabel(r'$\varepsilon$', fontsize = 20)
plt.ylabel('$E[\mu_{RMSE}]$', fontsize = 20)
plt.xticks(epsilon_vec[0: len(epsilon_vec)])
#plt.yticks(np.arange(0,3, step = 0.25))
#plt.ylim([0, 3]) 
plt.grid(True)
plt.legend(fontsize = 14, bbox_to_anchor=(0.5, 0.85), bbox_transform = plt.gcf().transFigure, loc = 'center', ncol = 3)
plt.show()

#### Variation of the RMSE standard deviation with $\varepsilon$ for different lattice densities

In [ ]:
def plot_expected_rmse_std_epsilon(epsilon_vec, expected_rmse_eps_std_netmech, expected_rmse_eps_std_adassp, lattice_density,\
                                   adassp = True):
    plt.semilogx(epsilon_vec, expected_rmse_eps_std_netmech, '--*', label = 'Net mechansim')
    if adassp:
        plt.plot(epsilon_vec, expected_rmse_eps_std_adassp, '--*', label = 'ADASSP')
        # plt.ylim([0, 1]) 
    plt.xlabel(r'$\varepsilon$', fontsize = 18)
   # plt.yticks(np.arange(0,5, step = 1))
   # plt.ylim([0, 5])
   #  plt.xticks(epsilon_vec)
    plt.ylabel(r'$\sigma_{RMSE}$ (density = ' + str(lattice_density) +  ' points)', fontsize = 18)
    plt.legend(fontsize = 14)
    plt.grid(True)
    plt.show()
        
for lattice_density in xcoordinate:
    # print ("DEBUG, expected_rmse_eps_std_netmech", expected_rmse_eps_std_netmech[lattice_density])
    plot_expected_rmse_std_epsilon(epsilon_vec[0: len(epsilon_vec)], expected_rmse_eps_std_netmech[lattice_density],\
                          expected_rmse_eps_std_adassp, lattice_density, adassp = True)

# Plot average RMSE for ADASSP for selected values of the lattice density on the same graph
plt.figure(1)
xcoordinate_range = [3, 4, 5]
for lattice_density in xcoordinate_range:
    plt.semilogx(epsilon_vec[0: len(epsilon_vec)], expected_rmse_eps_std_netmech[lattice_density], '--*',\
                 label = 'density = ' + str(lattice_density))
plt.plot(epsilon_vec[0: len(epsilon_vec)], expected_rmse_eps_std_adassp,'-*', label = 'ADASSP')
plt.xlabel(r'$\varepsilon$', fontsize = 20)
plt.ylabel(r'$\sigma_{RMSE}$', fontsize = 20)
plt.xticks(epsilon_vec[0: len(epsilon_vec)])
#plt.yticks(np.arange(0, 10, step = 1))
#plt.ylim([0, 10]) 
plt.grid(True)
plt.legend(fontsize = 14, bbox_to_anchor=(0.5, 0.85), bbox_transform = plt.gcf().transFigure, loc = 'center', ncol = 3)
plt.show()

#### Optimal utility

In [ ]:
print(expected_optimal_utilities)

In [ ]:
# Independent of epsilon, all the experiments return the same utility
plt.plot(xcoordinate, expected_optimal_utilities, 'r*', label = r'$E[opt(u)]$')
plt.plot(xcoordinate, max_utilities[23], 'cD', label = r'$opt(u)_{23}$')
plt.plot(xcoordinate, min_optimal_utilities, 'Xk', label =r'$\min(opt(u))$' )
plt.plot(xcoordinate, max_optimal_utilities, 'Xr', label =r'$\max(opt(u))$' )
plt.xticks(xcoordinate)
# plt.yticks(np.arange(-0.07, 0.0, step = 0.01))
plt.xlabel('Lattice density (points)', fontsize = 18)
plt.ylabel(r'$opt(u)$', fontsize = 18)
# plt.ylim([-0.07,0])
plt.grid(True)
plt.legend(fontsize = 13)
plt.show()

#### Expected minimum sample error for ADASSP vs netmechanism

In [ ]:
print(expected_min_predictive_errs_netmech)
print(expected_min_predictive_err_adassp)

In [ ]:
def plot_expected_min_rmses(xcoordinate, expected_min_predictive_errs_netmech, expected_min_predictive_err_adassp, epsilon):
    plt.plot(xcoordinate, expected_min_predictive_errs_netmech, 'bv', label = 'Net mechanism',  )
    plt.plot(xcoordinate, [expected_min_predictive_err_adassp]*len(xcoordinate), 'k^', label = 'ADASSP')
    plt.xlabel('Lattice density (points)', fontsize = 18)
    plt.ylabel(r'$\min$ RMSE ($\varepsilon =' + str(epsilon) + '$)', fontsize = 18)
    plt.xticks(xcoordinate)
    plt.legend(fontsize = 14)
    plt.grid(True)
    plt.show()
    
for epsilon in epsilon_vec:
    plot_expected_min_rmses(xcoordinate, expected_min_predictive_errs_netmech[epsilon], expected_min_predictive_err_adassp[epsilon], \
                   epsilon)

#### Minimum of min sample error for ADASSP vs netmechanism

### Save data for cross-experiment comparison

In [ ]:
data = {}
data['delta_opt_avg'] = delta_opt_avg
data['delta_opt_best'] = delta_opt_best
data['max_utilities'] = max_utilities
data['rmse_eps_avg_netmech'] = rmse_eps_avg_netmech
data['rmse_eps_avg_adassp'] = rmse_eps_avg_adassp

In [ ]:
filename = 'exp_vary_private_data'
with open (directory + filename, 'wb') as container:
    pickle.dump(data, container)

# Sanity checks

#### Calculate the raw stats and compare with output of complicated function

In [ ]:
helper_out = {key: [] for key in epsilon_vec}
means = {key: [] for key in epsilon_vec}
mins = {key: [] for key in epsilon_vec}
maxs = {key: [] for key in epsilon_vec}
for dataset_number in avg_samples_utility.keys():
    for epsilon in avg_samples_utility[dataset_number].keys():
        helper_out[epsilon].append(avg_samples_utility[dataset_number][epsilon])
for epsilon in helper_out.keys():
    means[epsilon] = np.mean(helper_out[epsilon], axis = 0)
    mins[epsilon] = np.min(helper_out[epsilon], axis = 0)
    maxs[epsilon] = np.max(helper_out[epsilon], axis = 0)
print (means[0.1])
print (expected_avg_utility[0.1])
for key in helper_out.keys():
    assert np.all(expected_avg_utility[key] == means[key])
    assert np.all(min_avg_utility[key] == mins[key])
    assert np.all(max_avg_utility[key] == maxs[key])